In [1]:
!pip install selenium
!pip install webdriver-manager 

### Importar as bibliotecas

In [2]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

### Vai baixar o driver referente a versão do nosso Chrome instalado

In [3]:
driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()))

In [4]:
url = 'https://www.fundamentus.com.br/resultado.php'
driver.get(url)

### Pegar a tabela do site fundamentus, e colocar no pandas

Passamos o thousand como . e decimal como , para não ter bug, pois nos EUA é ao contrário


In [5]:
local_tabela = '/html/body/div[1]/div[2]/table'

elemento = driver.find_element("xpath", local_tabela)

html_tabela = elemento.get_attribute("outerHTML")

tabela = pd.read_html(str(html_tabela), thousands = '.', decimal = ',')[0]

tabela

,Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,...,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a
0,IVTT3,0.00,0.00,0.00,0.000,"0,00%",0.000,0.00,0.00,0.0,...,0.00,"0,00%","0,00%",0.00,"0,00%","-0,40%",0.0,1.083050e+09,0.00,"20,67%"
1,MNSA4,0.47,0.00,0.00,0.000,"0,00%",0.000,0.00,0.00,0.0,...,0.00,"-208,15%","-362,66%",3.63,"-13,50%","145,70%",0.0,-9.105000e+06,-6.52,"-41,11%"
2,PMET3,0.00,0.00,0.00,0.000,"0,00%",0.000,0.00,0.00,0.0,...,0.00,"0,00%","0,00%",0.00,"0,00%","4,10%",0.0,-2.908630e+08,0.00,"37,74%"
3,MNSA3,0.42,0.00,0.00,0.000,"0,00%",0.000,0.00,0.00,0.0,...,0.00,"-208,15%","-362,66%",3.63,"-13,50%","145,70%",0.0,-9.105000e+06,-6.52,"-41,11%"
4,CSTB4,147.69,0.00,0.00,0.000,"0,00%",0.000,0.00,0.00,0.0,...,0.00,"40,85%","28,98%",2.60,"22,40%","20,11%",0.0,8.420670e+09,0.14,"31,91%"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
981,PRBC4,14.54,502.29,40.26,0.000,"0,00%",0.000,0.00,0.00,0.0,...,0.00,"0,00%","0,00%",0.00,"0,00%","8,02%",0.0,1.176410e+09,0.00,"-6,01%"
982,UBBR4,7.49,610.27,1.99,0.000,"0,00%",0.000,0.00,0.00,0.0,...,0.00,"0,00%","0,00%",0.00,"0,00%","0,33%",0.0,1.031720e+10,0.00,"10,58%"
983,UBBR11,14.75,1201.81,3.91,0.000,"0,00%",0.000,0.00,0.00,0.0,...,0.00,"0,00%","0,00%",0.00,"0,00%","0,33%",0.0,1.031720e+10,0.00,"10,58%"
984,UBBR3,18.00,1466.61,4.77,0.000,"0,00%",0.000,0.00,0.00,0.0,...,0.00,"0,00%","0,00%",0.00,"0,00%","0,33%",0.0,1.031720e+10,0.00,"10,58%"


### Pegamos os indicadores fundalistas de todas as empresas da bolsa e agora vamos construir a tabela final 

1: Setamos papel como indice

2: Selecionamos os dados que precisamos

In [6]:
tabela = tabela.set_index('Papel')

In [7]:
tabela = tabela[['Cotação', 'EV/EBIT', 'ROIC', 'Liq.2meses']]
tabela

,Cotação,EV/EBIT,ROIC,Liq.2meses
Papel,,,,
IVTT3,0.00,0.00,"0,00%",0.0
MNSA4,0.47,0.00,"-13,50%",0.0
PMET3,0.00,0.00,"0,00%",0.0
MNSA3,0.42,0.00,"-13,50%",0.0
CSTB4,147.69,0.00,"22,40%",0.0
...,...,...,...,...
PRBC4,14.54,0.00,"0,00%",0.0
UBBR4,7.49,0.00,"0,00%",0.0
UBBR11,14.75,0.00,"0,00%",0.0


### Tratar os textos

In [8]:
tabela['ROIC'] = tabela['ROIC'].str.replace("%", "")
tabela['ROIC'] = tabela['ROIC'].str.replace(".", "")
tabela['ROIC'] = tabela['ROIC'].str.replace(",", ".")
tabela['ROIC'] = tabela['ROIC'].astype(float)
tabela

,Cotação,EV/EBIT,ROIC,Liq.2meses
Papel,,,,
IVTT3,0.00,0.00,0.00,0.0
MNSA4,0.47,0.00,-13.50,0.0
PMET3,0.00,0.00,0.00,0.0
MNSA3,0.42,0.00,-13.50,0.0
CSTB4,147.69,0.00,22.40,0.0
...,...,...,...,...
PRBC4,14.54,0.00,0.00,0.0
UBBR4,7.49,0.00,0.00,0.0
UBBR11,14.75,0.00,0.00,0.0


### Fazendo filtros e criando o Rank

In [9]:
tabela = tabela[tabela['Liq.2meses'] > 1000000]

In [10]:
tabela = tabela[tabela['EV/EBIT'] > 0]
tabela = tabela[tabela['ROIC'] > 0]


In [11]:
tabela['ranking_ev_ebit'] = tabela['EV/EBIT'].rank(ascending = True)
tabela['ranking_roic'] = tabela['ROIC'].rank(ascending = False)
tabela['ranking_total'] = tabela['ranking_ev_ebit'] + tabela['ranking_roic']

tabela = tabela.sort_values('ranking_total')

tabela

,Cotação,EV/EBIT,ROIC,Liq.2meses,ranking_ev_ebit,ranking_roic,ranking_total
Papel,,,,,,,
PSSA3,25.74,0.21,60.76,3.407740e+07,1.0,1.0,2.0
PETR4,35.70,2.62,28.95,1.740010e+09,4.0,9.5,13.5
PETR3,38.76,2.77,28.95,4.776220e+08,5.0,9.5,14.5
UNIP6,71.54,4.50,38.68,1.601960e+07,14.0,3.0,17.0
WIZC3,5.12,1.92,24.18,4.441510e+06,3.0,16.0,19.0
...,...,...,...,...,...,...,...
CBAV3,3.79,82.23,0.53,1.649930e+07,184.0,186.0,370.0
PCAR3,3.46,42.60,0.24,3.903260e+07,181.0,189.0,370.0
AMAR3,2.18,111.35,0.58,3.197060e+06,187.0,185.0,372.0


In [12]:
tabela.head(10)

,Cotação,EV/EBIT,ROIC,Liq.2meses,ranking_ev_ebit,ranking_roic,ranking_total
Papel,,,,,,,
PSSA3,25.74,0.21,60.76,3.407740e+07,1.0,1.0,2.0
PETR4,35.70,2.62,28.95,1.740010e+09,4.0,9.5,13.5
PETR3,38.76,2.77,28.95,4.776220e+08,5.0,9.5,14.5
UNIP6,71.54,4.50,38.68,1.601960e+07,14.0,3.0,17.0
WIZC3,5.12,1.92,24.18,4.441510e+06,3.0,16.0,19.0
KEPL3,10.67,4.72,43.61,1.339680e+07,19.0,2.0,21.0
VLID3,16.58,4.09,20.64,7.216230e+06,9.5,22.0,31.5
FESA4,48.41,4.68,25.57,8.978620e+06,18.0,14.0,32.0
GOAU4,10.24,1.31,19.35,7.631950e+07,2.0,32.0,34.0
